### CSC478 Final Project: MovieLens
### Jun Tae Son |  Yuchen Wu | Tzu Hao Peng

In [23]:
import os
import math
import pandas as pd
import numpy as np
import sqlite3

# change the directory
os.chdir('C:\\Users\\rkfql\\Desktop\\Movielens-02')

In [24]:
# read rating data
rating_cols = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=rating_cols)
print(ratings.shape)
ratings.head()

(100000, 4)


,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [25]:
# read movie data
item_cols = ['movie_id','movie_title','release_date','video_release_date','IMDb_URL','unknown','Action','Adventure','Animation','Childrens','Comedy','Crime','Documentary','Drama','Fantasy','Film_Noir','Horror','Musical','Mystery','Romance','Sci_Fi','Thriller','War' ,'Western']
movies=pd.read_csv('u.item',sep='|',encoding='iso-8859-1', names=item_cols)
print(movies.shape)
movies.head()

(1682, 24)


,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [26]:
# read user data
user_cols = ['user_id','age','gender','occupation','zip_code']
users = pd.read_csv('u.user', sep='|', names=user_cols)
print(users.shape)
users.head()

(943, 5)


,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


# SQLite db

In [52]:
# database scheme
ratings_table = '''CREATE TABLE ratings
(
    user_id NUMBER(4),
    movie_id NUMBER(4),
    rating NUMBER(1) NOT NULL,
    timestamp NUMBER(10),
    
    CONSTRAINT rating_pk
        PRIMARY KEY (user_id, movie_id),

    CONSTRAINT tweet_fk1
        FOREIGN KEY (user_id)
        REFERENCES users(user_id),
        
    CONSTRAINT tweet_fk2
        FOREIGN KEY (movie_id)
        REFERENCES movies(movie_id)     
);
'''
    
movies_table = '''CREATE TABLE movies
(
    movie_id NUMBER(4),
    movie_title VARCHAR(30) NOT NULL,
    release_date VARCHAR(15),
    video_release_date VARCHAR(15),
    IMDb_URL VARCHAR(500),
    unknown NUMBER(1),
    Action NUMBER(1),
    Adventure NUMBER(1),
    Animation NUMBER(1),
    Childrens NUMBER(1),
    Comedy NUMBER(1),
    Crime NUMBER(1),
    Documentary NUMBER(1),
    Drama NUMBER(1),
    Fantasy NUMBER(1),
    Film_Noir NUMBER(1),
    Horror NUMBER(1),
    Musical NUMBER(1),
    Mystery NUMBER(1),
    Romance NUMBER(1),
    Sci_Fi NUMBER(1),
    Thriller NUMBER(1),
    War NUMBER(1),
    Western NUMBER(1),
    
    CONSTRAINT movies_pk
        PRIMARY KEY (movie_id)       
);
'''
    
users_table = '''CREATE TABLE users
(
    user_id NUMBER(4),  
    age NUMBER(2),
    gender VARCHAR(1),
    occupation VARCHAR(25),
    zip_code NUMBER(5),
    
    CONSTRAINT users_pk
        PRIMARY KEY (user_id)       
);
'''

In [10]:
conn = sqlite3.connect("movies.db")
cursor = conn.cursor()
cursor.execute('DROP TABLE IF EXISTS users;')
cursor.execute(users_table)

cursor.execute('DROP TABLE IF EXISTS movies;')
cursor.execute(movies_table)

cursor.execute('DROP TABLE IF EXISTS ratings;')
cursor.execute(ratings_table)

In [41]:
# change tables into numpy array
users_array = np.array(users)
movies_array = np.array(movies)
ratings_array = np.array(ratings)

In [40]:
# insert values into users table
for i in range(len(users_array)):
    try:
        cursor.execute("INSERT OR IGNORE INTO users VALUES (?,?,?,?,?);", 
                       [users_array[i][0], users_array[i][1], users_array[i][2], users_array[i][3], users_array[i][4]])
        conn.commit()
    except:
        print(movies_array[i])
        pass

In [50]:
# insert values into movies table
for i in range(len(movies_array)):
    try:
        #if movies_array[i][4]
        cursor.execute("INSERT OR IGNORE INTO movies VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);", 
                       [movies_array[i][0], movies_array[i][1], movies_array[i][2], movies_array[i][3], movies_array[i][4],
                       movies_array[i][5], movies_array[i][6], movies_array[i][7], movies_array[i][8], movies_array[i][9],
                       movies_array[i][10], movies_array[i][11], movies_array[i][12], movies_array[i][13], movies_array[i][14],
                       movies_array[i][15], movies_array[i][16], movies_array[i][17], movies_array[i][18], movies_array[i][19],
                       movies_array[i][20], movies_array[i][21], movies_array[i][22], movies_array[i][23]])
        conn.commit()
    except:
        print(movies_array[i])
        pass

In [51]:
# insert values into ratings table
for i in range(len(ratings_array)):
    try:
        cursor.execute("INSERT OR IGNORE INTO ratings VALUES (?,?,?,?);", 
                       [ratings_array[i][0], ratings_array[i][1], ratings_array[i][2], ratings_array[i][3]])
        conn.commit()
    except:
        print(movies_array[i])
        pass

In [ ]:
print("{} number of rows were successfully loaded in users table!".format(conn.execute("SELECT COUNT(*) FROM users").fetchall()[0][0]))
print("{} number of rows were successfully loaded in movies table!".format(conn.execute("SELECT COUNT(*) FROM movies").fetchall()[0][0]))
print("{} number of rows were successfully loaded in ratings table!".format(conn.execute("SELECT COUNT(*) FROM ratings").fetchall()[0][0]))

In [ ]:
# close
conn.close()